In [94]:
import sklearn
from surprise import Dataset
from surprise import evaluate
from surprise import KNNBasic
from surprise import Reader
import pandas as pd
import numpy as np
#from scipy.sparse import csr_matrix

In [95]:
triplet_data = pd.read_table('/Users/christinacampbell/downloads/10000.txt',header = None)
triplet_data.columns= ['user_id', 'song_id', 'listen_count']
song_data = pd.read_csv('/Users/christinacampbell/downloads/song_data.csv')
songs = song_data.values


In [96]:
popularity_threshold = 26
triplet_data = triplet_data.query('listen_count >= @popularity_threshold')
triplet_data.shape


(20509, 3)

In [97]:
#sort data by users
new_data=triplet_data.sort_values(by=['user_id']).reset_index(drop=True)

In [98]:
#shows the range of listen counts
print(new_data['listen_count'].describe())

count    20509.000000
mean        45.663026
std         37.053883
min         26.000000
25%         30.000000
50%         36.000000
75%         49.000000
max       2213.000000
Name: listen_count, dtype: float64


In [99]:
#check to see if you can limit the range of ratings
len(new_data.loc[new_data['listen_count']<=130])
#130 would allow for me to turn the ratings into 1-5 scale = easier to compare and will make the data more clustered

20066

In [100]:
#limit the data to only ratings less than 130
new_data = new_data[new_data.listen_count <=130]
#divide listen counts by 26 to get it into a 1-5 rating
new_data['listen_count'] = new_data['listen_count'].apply(lambda x: x/26)

In [101]:
#find new max and min
print(new_data['listen_count'].describe())

count    20066.000000
mean         1.615479
std          0.707456
min          1.000000
25%          1.115385
50%          1.346154
75%          1.846154
max          5.000000
Name: listen_count, dtype: float64


In [102]:
#drop users that only have 1 appearance
new_data['is_duplicate']=  new_data.duplicated(['user_id'], keep =False).values
new_data = new_data.query('is_duplicate == True')
new_data=new_data.drop(['is_duplicate'], axis=1).reset_index(drop=True)


In [103]:
#drop songs that only have 1 rating
new_data['song_dups'] = new_data.duplicated(['song_id'], keep =False).values
new_data= new_data.query('song_dups== True')
new_data = new_data.drop(['song_dups'], axis = 1).reset_index(drop=True)
new_data.shape

(12063, 3)

In [104]:
#limit data to have only songs that are listened to by more than 7 users
#helps increase the number of predictions that can be made because tightens the clusters
#doing this for users leads to worse performance because it limits the amount of comparable data
new_data = new_data.groupby('song_id').filter(lambda x : (x['song_id'].count()>=7).any())

In [105]:
new_data.shape

(6238, 3)

In [106]:
#remove 1 rating for each user in order to use it to evaluate the accuracy of my predictions
#I now have a rating to compare the given prediction with for a selection of the data
new_data['find_dup']= new_data.duplicated(['user_id'], keep='first').values
new_data['find_nondup'] = new_data.duplicated(['user_id'], keep=False).values
testing_ratings = new_data[(new_data.find_dup==False) & (new_data.find_nondup==True)]
new_data =new_data[(new_data.find_dup==True) | (new_data.find_nondup==False)]


In [107]:
len(testing_ratings)

1792

In [108]:
#remove the unnessecary columns and reset the index
new_data = new_data.drop(['find_dup','find_nondup'],axis=1).reset_index(drop=True)


In [109]:
#load the data into the surprise dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_data[['user_id', 'song_id', 'listen_count']],reader = reader)
#create the train set
trainset = data.build_full_trainset()

In [110]:
#initialize the model
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [111]:
#fit the trainset using the knn model
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [112]:
# create a list of predictions for each of the rows in the testing set I collected above. 
#This will be used to allow me to accurately see how successful the model is, because I have the given ratings by these users
from collections import defaultdict
predictions=[]

test_rates = testing_ratings.values
for i in range(len(test_rates)-1):
    predictions.append(knn.predict(test_rates[i][0],test_rates[i][1], test_rates[i][2]))
    

In [113]:
import math
#calculates average difference between the actual rating and the predicted rating
diff=[]
exact=[]
len(predictions)
for i in range(len(predictions)-1):
    #val= (predictions[i][2] - predictions[i][3])/2213
    val = abs(predictions[i][2] - predictions[i][3])
    diff.append(val)
    
avg_difference=sum(diff)/len(diff)
avg_difference

0.5566769704159487

In [114]:
exact=[]
#calculates the percent of possible predictions where the difference was less than the average difference
for i in range(len(predictions)-1):
    val= (predictions[i][2] - predictions[i][3])
    #val = (abs(Eval[i][2] - Eval[i][3]))
    if val<avg_difference:
        exact.append(val)
len(exact)/len(predictions)*100

82.41206030150754

In [115]:
#rmse for all predictions (including those that were impossible and therefore just given the mean score)
#interestingly, the rmse is better for all predictions, which shows assigning the mean rating proves to be successful
from surprise import accuracy
accuracy.rmse(predictions,verbose =True)

RMSE: 0.7896


0.7895896072791613

In [116]:
# a function to return the n top recommendation for each user based off of the predictions; 
#as of right now, it returns just the song id of each, but I plan to fix it to return the song name as well as the difference in the ratings
from collections import defaultdict
def top_predictions(prediction,n):
    top = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top[uid] = user_ratings[:n]

    return top

In [117]:
##originally was 54 million tests, had to find a way to limit the number of songs without ratings
#build_anti_testset(): makes a testset of all of the songs that arent rated by a user
#AKA every song that a user hasn't listened to will be added to the test set for it to calculate a prediction for it
testset = trainset.build_anti_testset()
    

In [118]:
len(testset)

1145978

In [119]:
#calculates all of the predictions
all_predictions = knn.test(testset)


In [120]:
#creates a list of all of the top predictions for each user
top = top_predictions(all_predictions,1)

In [121]:
#returns a list of all of the users and their predictions;
for uid, user_ratings in top.items():
    new_list = (uid, [iid for (iid, _) in user_ratings])
    for song in songs:
        if (song[0]== new_list[1][0]):
            print("Recommendation for " + new_list[0] + ":")
            print( "\'" + song[1] +  "\'"+ ' by ' + song[3]+ "\n")


('0031572620fa7f18487d3ea22935eb28410ecc4c', ['SOLDJLX12A6D4F9A78'])
Recommendation for 0031572620fa7f18487d3ea22935eb28410ecc4c:
'Last Cup Of Sorrow' by Faith No More

('0091e0326c4c034cc04be6454742912845740a1f', ['SOQQAAQ12A67ADE34D'])
Recommendation for 0091e0326c4c034cc04be6454742912845740a1f:
'Welcome To The Black Parade (Album Version)' by My Chemical Romance

('00a443baf550f4bbdd974ba73720abf2759166f3', ['SOUNZHU12A8AE47481'])
Recommendation for 00a443baf550f4bbdd974ba73720abf2759166f3:
'I CAN'T GET STARTED' by Ron Carter

('00bc2c7795aeec17772d5e3be3bb0bd23f1167b9', ['SOMHLHX12A58A7AA5B'])
Recommendation for 00bc2c7795aeec17772d5e3be3bb0bd23f1167b9:
'You Learn (Album Version)' by Alanis Morissette

('00d941401206a4b252886d059dc9cf37096a68ee', ['SOAUWYT12A81C206F1'])
Recommendation for 00d941401206a4b252886d059dc9cf37096a68ee:
'Undo' by Björk

('00d94de9e6f82fa1527da8091d56d2cb3b8b30de', ['SOTUARP12A8C13CB54'])
Recommendation for 00d94de9e6f82fa1527da8091d56d2cb3b8b30de:
'Rio' b

('06cfb3cad4799cc7b728c9965056d9114817f605', ['SOLHTJY12A58A7AFE7'])
Recommendation for 06cfb3cad4799cc7b728c9965056d9114817f605:
'Go Skate! (Possessed To Skate '97)' by Suicidal Tendencies

('06d14f01b1d51acb598c505acbb5c4fb9b22970e', ['SOSXLTC12AF72A7F54'])
Recommendation for 06d14f01b1d51acb598c505acbb5c4fb9b22970e:
'Revelry' by Kings Of Leon

('06d6f909647cd9f13f296c0d6f2b5b95202d7f74', ['SOPSOHT12A67AE0235'])
Recommendation for 06d6f909647cd9f13f296c0d6f2b5b95202d7f74:
'Almaz' by Randy Crawford

('0701b97518ec73b56dad803496de3382047bc5f3', ['SOBONKR12A58A7A7E0'])
Recommendation for 0701b97518ec73b56dad803496de3382047bc5f3:
'You're The One' by Dwight Yoakam

('07068d0fb0eb5d1d5ff275f71ead1387b31a3a5e', ['SOXZOMB12AB017DA15'])
Recommendation for 07068d0fb0eb5d1d5ff275f71ead1387b31a3a5e:
'Word Up!' by Cameo

('071a964898dc8f6d4aef983fa45bbea65ab29168', ['SONDKOF12A6D4F7D70'])
Recommendation for 071a964898dc8f6d4aef983fa45bbea65ab29168:
'Recado Falado (Metrô Da Saudade)' by Alceu Vale

('0de9ced79f560aa0e51c413fc54ec5d39f1e7386', ['SOZEBAZ12AF72A80C8'])
Recommendation for 0de9ced79f560aa0e51c413fc54ec5d39f1e7386:
'Voices On A String (Album Version)' by Thursday

('0dea223b7ee4abb5fbbde2b47e363e5a4bc8b92f', ['SOFKABN12A8AE476C6'])
Recommendation for 0dea223b7ee4abb5fbbde2b47e363e5a4bc8b92f:
'Just Dance' by Lady GaGa / Colby O'Donis

('0debe21593ddb30df5ea7fa28f875f96e75cc180', ['SOYTDBL12A6D4F8148'])
Recommendation for 0debe21593ddb30df5ea7fa28f875f96e75cc180:
'What We Do (Explicit) (Feat. Memphis Bleek)' by Sauce Money Featuring Memphis Bleek

('0e2b9404952e8fea06e41d55d18181389a747da3', ['SOSJSSU12A6D4F8F41'])
Recommendation for 0e2b9404952e8fea06e41d55d18181389a747da3:
'Victoria (LP Version)' by Old 97's

('0e2f6624a9205bbfec266299bdfcf01c1fbef6d9', ['SOGLWGG12A6D4F8046'])
Recommendation for 0e2f6624a9205bbfec266299bdfcf01c1fbef6d9:
'The Want Ad' by Pandora's Box

('0e402e03948973ef4991b18ae03d192e8fcc8b16', ['SOMCMKG12A8C1347BF'])
Recommendation for 0e402e03948973

('1390a58ef4f715c78c59332a805658725f2e7f05', ['SOPPROJ12AB0184E18'])
Recommendation for 1390a58ef4f715c78c59332a805658725f2e7f05:
'Pursuit Of Happiness (nightmare)' by Kid Cudi / MGMT / Ratatat

('13a4fdbc5e94c207170b19a9506ee1ba9d248c84', ['SOAXGDH12A8C13F8A1'])
Recommendation for 13a4fdbc5e94c207170b19a9506ee1ba9d248c84:
'Dog Days Are Over (Radio Edit)' by Florence + The Machine

('13ac51ac39c657a0034e7253499bee84fe6d75ed', ['SOAUWYT12A81C206F1'])
Recommendation for 13ac51ac39c657a0034e7253499bee84fe6d75ed:
'Undo' by Björk

('13f974c5f9fd72385f91d1d3616590ff50c21f82', ['SOKYHYL12A8C13D79D'])
Recommendation for 13f974c5f9fd72385f91d1d3616590ff50c21f82:
'It Was A Good Day' by Ice Cube

('14014043846cd3920ad9b992a7efff16cdd73fc9', ['SOPUCYA12A8C13A694'])
Recommendation for 14014043846cd3920ad9b992a7efff16cdd73fc9:
'Canada' by Five Iron Frenzy

('1404d1add516dc73853c93ac5a724c143c72a310', ['SOKOSPK12A8C13C088'])
Recommendation for 1404d1add516dc73853c93ac5a724c143c72a310:
'Call It Off (A

('1b9db14239b65febdd95bc0ca1a4a5ace307cab8', ['SOHFNKO12AB017C772'])
Recommendation for 1b9db14239b65febdd95bc0ca1a4a5ace307cab8:
'Kiss With A Fist' by Florence + The Machine

('1c03bdf1b0b7cf6031e510e4bd9e34e9040a06e7', ['SOJUEFU12AC3DF97C2'])
Recommendation for 1c03bdf1b0b7cf6031e510e4bd9e34e9040a06e7:
'Modern world' by Modern Lovers

('1c0a43b2c8fa99ed4a1ffabec4af78c250922943', ['SOVDSJC12A58A7A271'])
Recommendation for 1c0a43b2c8fa99ed4a1ffabec4af78c250922943:
'Ain't Misbehavin' by Sam Cooke

('1c3a0cc0d614eea1bbbc80d0f24a9e9d790bd48a', ['SOBOSGD12A6D4F75C0'])
Recommendation for 1c3a0cc0d614eea1bbbc80d0f24a9e9d790bd48a:
'Kelma' by Rachid Taha

('1c529e25ce0d76bfc188633fb5515c6205efe863', ['SOOFYTN12A6D4F9B35'])
Recommendation for 1c529e25ce0d76bfc188633fb5515c6205efe863:
'Représente' by Alliance Ethnik

('1c6d8193a0c1dbcd678c807855a08f1f8678a16a', ['SOHALKJ12AB017FCC2'])
Recommendation for 1c6d8193a0c1dbcd678c807855a08f1f8678a16a:
'Speak' by Godsmack

('1c7211119059016341a081b056a4

...

('de3de2c484ccb7cd28eb0f8f6918a6d7b70f924e', ['SOUJVIT12A8C1451C1'])
Recommendation for de3de2c484ccb7cd28eb0f8f6918a6d7b70f924e:
'Savior' by Rise Against

('de8e36e4ed6b7ea7de96269a87b36210c561d0a1', ['SOBFRNE12A6D4F7995'])
Recommendation for de8e36e4ed6b7ea7de96269a87b36210c561d0a1:
'Le Courage Des Oiseaux' by Dominique A

('deb2cb3d8418888e3a76091d7387fcb19951d572', ['SOVIZNF12AF72A710A'])
Recommendation for deb2cb3d8418888e3a76091d7387fcb19951d572:
'The Big Gundown' by The Prodigy

('debb8ee39a0becc555482a99432ef58a3ac28b30', ['SODQZCY12A6D4F9D11'])
Recommendation for debb8ee39a0becc555482a99432ef58a3ac28b30:
'El Cuatrero' by Miguel Calo

('debd74c1999b195eeddf09bc0af751c93a52c205', ['SOLFXKT12AB017E3E0'])
Recommendation for debd74c1999b195eeddf09bc0af751c93a52c205:
'Fireflies' by Charttraxx Karaoke

('dec745c42b149c8549cee4cd7a30b7cae64e1809', ['SOLGLUC12AB018A8FA'])
Recommendation for dec745c42b149c8549cee4cd7a30b7cae64e1809:
'Frisch und g'sund' by Die Mooskirchner

('df19f442d8a

('e6584edb6db73c921dd58c6f658df12e98d77157', ['SOZCDWG12A6D4F81E1'])
Recommendation for e6584edb6db73c921dd58c6f658df12e98d77157:
'A Beggar On A Beach Of Gold' by Mike And The Mechanics

('e67ed8ade96b9073ff1e195f8fa5e35d9011f9dd', ['SOXLOQG12AF72A2D55'])
Recommendation for e67ed8ade96b9073ff1e195f8fa5e35d9011f9dd:
'Unite (2009 Digital Remaster)' by Beastie Boys

('e6851c97674708cf42def7308f92d362946f4170', ['SONETQG12AF72A16FF'])
Recommendation for e6851c97674708cf42def7308f92d362946f4170:
'Give It Back' by Polly Paulusma

('e69c2e07fe9e333ccc796bbfbfd495d37c39803d', ['SODGVGW12AC9075A8D'])
Recommendation for e69c2e07fe9e333ccc796bbfbfd495d37c39803d:
'Somebody To Love' by Justin Bieber

('e69c2e07fe9e333ccc796bbfbfd495d37c39803d', ['SODGVGW12AC9075A8D'])
Recommendation for e69c2e07fe9e333ccc796bbfbfd495d37c39803d:
'Somebody To Love' by Justin Bieber

('e6a19b40faa89ac46c24b03d8aa120e723eee15e', ['SOUDLVN12AAFF43658'])
Recommendation for e6a19b40faa89ac46c24b03d8aa120e723eee15e:
'Make 

('ed82cbf95d6d333f984a43978f5425bdbd0d2e28', ['SOXKOIY12A8C13C1EA'])
Recommendation for ed82cbf95d6d333f984a43978f5425bdbd0d2e28:
'Take Me Out' by Franz Ferdinand

('edc78b433b0eed6e1ebe787718a709662c531430', ['SOQZYQH12A8AE468E5'])
Recommendation for edc78b433b0eed6e1ebe787718a709662c531430:
'Better To Reign In Hell' by Cradle Of Filth

('edf1c26f90f5ef43d1eb11c12215adbaec1d0417', ['SOPUCYA12A8C13A694'])
Recommendation for edf1c26f90f5ef43d1eb11c12215adbaec1d0417:
'Canada' by Five Iron Frenzy

('ee14fb56084ea6b5435fd732e531809044085dac', ['SOVDSJC12A58A7A271'])
Recommendation for ee14fb56084ea6b5435fd732e531809044085dac:
'Ain't Misbehavin' by Sam Cooke

('ee153c57b5f092a8d4a82c91dbc08d8024decb16', ['SOITRTA12A6D4F8261'])
Recommendation for ee153c57b5f092a8d4a82c91dbc08d8024decb16:
'Ride The Lightning' by Metallica

('ee651111cac96b2f0cba2a5fdbb33a4ba18bba3b', ['SOUDLVN12AAFF43658'])
Recommendation for ee651111cac96b2f0cba2a5fdbb33a4ba18bba3b:
'Make Love To Your Mind' by Bill Withers



('f615e527cdcc96de20c90faccc62d1c634304670', ['SOANOQW12A58A793D2'])
Recommendation for f615e527cdcc96de20c90faccc62d1c634304670:
'Cold Blooded (Acid Cleanse)' by The fFormula

('f652a16f171479e6443f99c9eb600c2c963f30af', ['SOOEGCA12A6D4F8B80'])
Recommendation for f652a16f171479e6443f99c9eb600c2c963f30af:
'Above The Clouds (Edited)' by Gang Starr/Inspectah Deck

('f6ad026a9d9498de5414beed42b687fae1521560', ['SOANQFY12AB0183239'])
Recommendation for f6ad026a9d9498de5414beed42b687fae1521560:
'Uprising' by Muse

('f6b657bc11b7d2312c13dce4ccc769dc835a2306', ['SOUFTBI12AB0183F65'])
Recommendation for f6b657bc11b7d2312c13dce4ccc769dc835a2306:
'Invalid' by Tub Ring

('f6dcd5180d7219559cdcc30f121b2b431cd7c4e1', ['SOWCKVR12A8C142411'])
Recommendation for f6dcd5180d7219559cdcc30f121b2b431cd7c4e1:
'Use Somebody' by Kings Of Leon

('f6dcd5180d7219559cdcc30f121b2b431cd7c4e1', ['SOWCKVR12A8C142411'])
Recommendation for f6dcd5180d7219559cdcc30f121b2b431cd7c4e1:
'Use Somebody' by Kings Of Leon

('f6e8

('fcc8017eca88158edb842b1f3fbcd633baa58e92', ['SOQCXSG12A8C13949F'])
Recommendation for fcc8017eca88158edb842b1f3fbcd633baa58e92:
'Fortunate Son (Live)' by Dropkick Murphys

('fcdedf71f739bd09f1bac19772e4afa927c8e37a', ['SONWUZV12AB0180BAD'])
Recommendation for fcdedf71f739bd09f1bac19772e4afa927c8e37a:
'Reason I'm Alive (Explicit)' by Drowning Pool

('fceced62e19ebb339920a2c17b98119ef0012d08', ['SOMCAFM12A58A7B024'])
Recommendation for fceced62e19ebb339920a2c17b98119ef0012d08:
'Who Can Compare' by Foolish Things

('fd08edfbba6799cb59641c0bb126a3ecda1d0bd7', ['SOVMGXI12AF72A80B0'])
Recommendation for fd08edfbba6799cb59641c0bb126a3ecda1d0bd7:
'Hey Mama' by Black Eyed Peas

('fd23113dcfb9f6acb2b4c49e38ad27d1c5b16dd6', ['SOBONKR12A58A7A7E0'])
Recommendation for fd23113dcfb9f6acb2b4c49e38ad27d1c5b16dd6:
'You're The One' by Dwight Yoakam

('fd74df82ab5be813b5530912e22a81f7ebafa677', ['SOADJQJ12A8C141D38'])
Recommendation for fd74df82ab5be813b5530912e22a81f7ebafa677:
'Halo' by Beyoncé

('fd78

In [31]:
#rmse of all predictions

accuracy.rmse(all_predictions,verbose = True)

RMSE: 0.2316


0.23156620029398656

In [ ]:
#notice that the accuracy is much better with more data and predictions